# **Dự đoán nồng độ CO2 trong bầu khí quyền**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **I. Đọc và xử lý dữ liệu**

Dataset được cung cấp bởi tổ chức mang tên **Scripps CO2 Program** mang tên:


> [Atmospheric CO2 Data](https://https://scrippsco2.ucsd.edu/data/atmospheric_co2/primary_mlo_co2_record.html)

> Đây là dataset về nồng độ CO₂ có trong khí quyển trung bình hàng tháng theo đơn vị (triệu ppm), được thu thập tại Đài quan sát khí hậu ***Mauna Loa*** ở Hawaii. Dự liệu được tổng hợp liên tục từ năm 1958 cho đến nay và Thể hiện mức độ CO₂ đang rất nhanh trong khí quyển.


In [2]:
#Thư viện đọc file csv
import pandas as pd 
import numpy as np

#truy cập directory
import os 

#Loại cảnh báo không cần thiết
import warnings

warnings.simplefilter("ignore")

df = pd.read_csv("/content/drive/MyDrive/RA_n_PA_Project/monthly_in_situ_co2_mlo.csv",
                  header=None,
                  skiprows=57, # Bỏ qua phần credit trong file CSV
                  usecols=[3, 4], # CHỉ lấy cột Date và CO2
                  na_values='-99.99', #Giá trị NaN trong file là -99.99
                  dtype=np.float64) #chuyển sang float64 phù hợp cho tf

#Định nghĩa cột
df.columns = ["Date", "CO2"]
df.head()

,Date,CO2
0,1958.0411,NaN
1,1958.1260,NaN
2,1958.2027,315.71
3,1958.2877,317.45
4,1958.3699,317.51


**1.1 Phát hiện và loại bỏ các ô có chứa giá trị rỗng (NaN value)**

In [3]:
print(df[df.isnull().any(axis=1)])
df.dropna(inplace=True)

          Date  CO2
0    1958.0411  NaN
1    1958.1260  NaN
5    1958.4548  NaN
9    1958.7890  NaN
73   1964.1257  NaN
74   1964.2049  NaN
75   1964.2896  NaN
765  2021.7890  NaN
766  2021.8740  NaN
767  2021.9562  NaN


In [4]:
print("Số dòng sau khi xóa NaN: ", df.shape[0])

Số dòng sau khi xóa NaN:  758


**2. Mô tả phân bố dữ liệu trong dataset**

In [5]:
df.describe()

,Date,CO2
count,758.000000,758.000000
mean,1990.140400,356.615607
std,18.284363,29.553204
min,1958.202700,313.210000
25%,1974.391125,329.805000
50%,1990.164350,353.470000
75%,2005.935650,380.530000
max,2021.706800,418.950000


Mô tả dự liệu trong project sẽ sử dụng [bokeh](https://bokeh.org/) là một thư viện cải tiến vượt trội hơn so với matplot với khả năng tương tác với người lập trình với các tính năng mới bao gồm:
*   Nắm kéo biểu đô đến điểm cần quan sát
*   Zoom biểu đồ
*   Xoay biểu đồ,..



In [6]:
from tqdm.notebook import tqdm
import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.models
from IPython.display import display, HTML

bokeh.io.output_notebook(hide_banner=True)

# Vẽ Dữ liệu với X là thời gian và y là nồng độ CO2
fig = bokeh.plotting.figure(
    width=800, height=400, 
    x_range=(1958, 2022), y_range=(310, 420))

fig.xaxis.axis_label = 'Thời gian (Date)'
fig.yaxis.axis_label = 'Nông độ CO₂ (triệu ppm)'

fig.add_layout(bokeh.models.Title(
    text='Các điểm dữ liệu quan sát được tại đài khí tượng Hawaii',
    text_font_style="italic"), 'above')

fig.add_layout(bokeh.models.Title(
    text='Nồng độ CO2 trên khí quyền theo tháng từ năm 1958-2021', 
    text_font_size="15pt"), 'above')

fig.circle( df.Date, df.CO2, legend_label='Các quan sát nồng độ CO2', size=1.5, line_color='midnightblue')
fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)

**Chia tâp test và train data**, trong đó:

*   Training data là tất cả các dòng từ năm 1958 ->2012
*   Testing data là tất cả các dòng trước sau 2012 -> nay



In [7]:
# Chia tập dataset thành 2 tập X observed và X* để dự đoán
date_split_predict = 2012
X_train = df[df.Date < date_split_predict]
print('Sử dụng {} dòng trước năm 2012 cho observed set'.format(len(X_train)))
X_test = df[df.Date >= date_split_predict]
print('Sử dụng {} dòng trước sau năm 2012 cho test set'.format(len(X_test)))

Sử dụng 641 dòng trước năm 2012 cho observed set
Sử dụng 117 dòng trước sau năm 2012 cho test set


# **II. Modeling**

In [8]:
# Imports các thư viện hỗ trợ xác suất trong tensorflow
import tensorflow as tf
import tensorflow_probability as tfp
# Định nghĩa tên viết tắt cho các hàm thường dùng
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels
np.random.seed(42)
tf.random.set_seed(42)

**2.1.Định nghĩa hàm Mean() cho các giá trị trung bình tại các điểm huấn luyện dưới dạng một Hằng (constant)** 

In [9]:
observations_mean = tf.constant(
    [np.mean(X_train.CO2.values)], dtype=tf.float64)

mean_kernel = lambda _: observations_mean

**2.2 Xây dựng hàm Kernel kết hợp**

**2.2.1 Định nghĩa các hyper-parameter cho quá trình huấn luyện:**

*   Tuân theo phân phối log chuẩn và luôn dương với giá trị khởi tạo ngẫu nhiên 5.0 hoặc 10.0
*   Kiểu float 64 (giảm khả năng lỗi khi tính phân rã Cholesky)


In [10]:
# Định nghĩa điều kiện luôn dương cho các siêu tham số
constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())

**2.2.2 Định nghĩa các tham số cho hàm Exponentiated Quadratic Kernel**

In [11]:
# Định nghĩa biên độ amplitude
exp_amplitude = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, 
    dtype=np.float64,name='exp_amplitude')

# Định nghĩa khoảng cách length_scale
exp_length_scale = tfp.util.TransformedVariable(
    initial_value=10., bijector=constrain_positive, 
    dtype=np.float64,name='exp_length_scale')

# Xây dựng hàm Exponentiated Quadratic Kernel
exp_kernel = tfk.ExponentiatedQuadratic(
    amplitude=exp_amplitude, length_scale=exp_length_scale)

**2.2.4 Định nghĩa các tham số số cho hàm Rational Quadratic kernel:**

In [12]:
# Các hyper- parameter của Rational Quadratic kernel:
# Định nghĩa biên độ amplitude
rational_amplitude = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, 
    dtype=np.float64,name='rational_amplitude')

# Định nghĩa khoảng cách length scale
rational_length_scale = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, 
    dtype=np.float64,name='rational_length_scale')

# Định nghĩa giá trị alpha scale-mixture (alpha > 0)
rational_scale_mixture = tfp.util.TransformedVariable(
    initial_value=1., bijector=constrain_positive, 
    dtype=np.float64,name='rational_scale_mixture')

#Xây dựng hàm Rational Quadratic kernel
rational_kernel = tfk.RationalQuadratic(
    amplitude=rational_amplitude,
    length_scale=rational_length_scale,
    scale_mixture_rate=rational_scale_mixture)

**2.2.3 Định nghĩa các tham số  cho hàm local periodic kernel:**

In [13]:
# Định nghĩa biên độ amplitude của hàm periodic.
periodic_amplitude = tfp.util.TransformedVariable(
    initial_value=5.0, bijector=constrain_positive, 
    dtype=np.float64,name='periodic_amplitude')

# Định nghĩa khoảng cách lengthscale của hàm periodic.
periodic_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=
    constrain_positive, dtype=np.float64,
    name='periodic_length_scale')

# Định nghĩa chu kỳ (period) của hàm periodic.
periodic_period = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, 
    dtype=np.float64,name='periodic_period')

# Định nghĩa khoảng cách lengthscale của hàm Exponentiated.
periodic_exp_length_scale = tfp.util.TransformedVariable(
    initial_value=1.0, bijector=constrain_positive, 
    dtype=np.float64,name='periodic_exp_length_scale')

# Xây dựng  hàm kernel local periodic kernel
# bằng phép and(tích) 2 kernel
local_periodic_kernel = (
    #Hàm periodic kernel
    tfk.ExpSinSquared(
        amplitude=periodic_amplitude, 
        length_scale=periodic_length_scale,
        period=periodic_period) * 
    # nhân với Hàm ExponentiatedQuadratic kernel
    tfk.ExponentiatedQuadratic(
        length_scale=periodic_exp_length_scale))

 **2.2.5 Tổng hợp hàm kernel tổng** từ 3 hàm kernel vừa định nghĩa bên trên:

In [14]:
# tổng tất cả 3 kernels thành một kernel:
kernel = (exp_kernel + local_periodic_kernel + rational_kernel)

**2.3 TỔng hợp các tham số huấn luyện mô hình cần thiết:**

In [15]:
# Định nghĩa noise variance cho các quan sát huấn luyện
observation_noise_variance = tfp.util.TransformedVariable(
    initial_value=1, bijector=constrain_positive, 
    dtype=np.float64,name='observation_noise_variance')

#Tổng hợp lại tất cả các tham số huấn luyện đã tạo
trainable_variables = [v.variables[0] for v in [
    #Các hyper-parameter của ExponentiatedQuadratic
    exp_amplitude,
    exp_length_scale,
    #Các hyper-parameter của Rational Quadratic kernel
    rational_amplitude,
    rational_length_scale,
    rational_scale_mixture,
    #Các hyper-parameter của local periodic kernel
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_exp_length_scale,
    #Độ nhiễu của các quan sát.
    observation_noise_variance
]]

Cấu hình batch size và thông tin của batch đó

In [16]:
#Kích thước batch
batch_size = 128

#Định nghĩa các batch dựa trên batch size
batched_dataset = (
    tf.data.Dataset.from_tensor_slices(
    (X_train.Date.values.reshape(-1, 1),X_train.CO2.values))
    .shuffle(buffer_size=len(X_train)).repeat(count=None).batch(batch_size))

**2.4 Định nghĩa hàm loss**


> Sử dụng **negative log likelihood**



In [17]:
@tf.function(autograph=False, experimental_compile=False)
def gp_loss_fn(index_points, observations):
# Hàm loss  negative-log-likelihood cho Gaussian process
    gp = tfd.GaussianProcess(
        mean_fn=mean_kernel,
        kernel=kernel,
        index_points=index_points,
        observation_noise_variance=observation_noise_variance)
    
    negative_log_likelihood = -gp.log_prob(observations)
    return negative_log_likelihood

**2.5 Tiến hành huấn luyện mô hình:**

In [18]:
from itertools import islice
import collections
# Sử dụng Adam optimizer với learning rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Lưu Negative Log-Likelihood (NLL) của cả batch
batch_loss = [] 

# Lưu Negative Log-Likelihood (NLL) của cả data
Loss_hist = [] 

#số lần train
iter=10000

for i, (index_points_batch, observations_batch) in tqdm(
        enumerate(islice(batched_dataset, iter)), total=iter):
    # Tiến hàn huấn luyện cho từng batch
    with tf.GradientTape() as tape:
        loss = gp_loss_fn(index_points_batch, observations_batch)
    grads = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    batch_loss.append((i, loss.numpy()))
    # Đánh giá trên tất cả các quan sát
    if i % 100 == 0:
        # Đánh giá trên tất cả các dữ liệu đã quan sats quan sát
        ll = gp_loss_fn(
            index_points=X_train.Date.values.reshape(-1, 1),
            observations=X_train.CO2.values)
        Loss_hist.append((i, ll.numpy()))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [19]:
# Lập đồ thị loss qua các lần train
fig = bokeh.plotting.figure(
    width=600, height=350, 
    x_range=(0, iter), y_range=(50, 200))
fig.add_layout(bokeh.models.Title(
    text='Negative Log Likelihood(loss) trong training', 
    text_font_size="14pt"), 'above')
fig.xaxis.axis_label = 'số lần train'
fig.yaxis.axis_label = 'loss của batch'
# Biểu diễn loss của các batch
fig.line(
    *zip(*batch_loss), legend_label='Batch data (loss)',
    line_width=2, line_color='lightblue')
# Biểu diễn loss của toàn bộ Training data
fig.extra_y_ranges = {
    'fig1ax2': bokeh.models.Range1d(start=130, end=250)}
fig.line(
    *zip(*Loss_hist), legend_label='Training data (loss)',
    line_width=2, line_color='red', y_range_name='fig1ax2')
# Thêm layout cho cột phải của toàn bộ loss
fig.add_layout(bokeh.models.LinearAxis(
    y_range_name='fig1ax2', axis_label='loss của Training data'), 'right')
fig.legend.location = 'top_right'

fig.toolbar.autohide = True
bokeh.plotting.show(fig)

In [20]:
# Trả về kết quả các Hyper-parameter sau training
variables = [
    #Các hyper-parameter của ExponentiatedQuadratic
    exp_amplitude,
    exp_length_scale,
    #Các hyper-parameter của Rational Quadratic kernel
    rational_amplitude,
    rational_length_scale,
    rational_scale_mixture,
    #Các hyper-parameter của local periodic kernel
    periodic_amplitude,
    periodic_length_scale,
    periodic_period,
    periodic_exp_length_scale,
    #Độ nhiễu của các quan sát.
    observation_noise_variance
]
#Chuyển list sang dataframe và in ra màn hình:
data = list([(var.variables[0].name[:-2], var.numpy()) for var in variables])
df_variables = pd.DataFrame(
    data, columns=['Hyperparameters', 'Value'])
display(HTML(df_variables.to_html(
    index=False, float_format=lambda x: f'{x:.4f}')))

Hyperparameters,Value
exp_amplitude,126.63614629671818
exp_length_scale,99.05400983040423
rational_amplitude,1.0166016020749205
rational_length_scale,1.3743845291890049
rational_scale_mixture,0.10211038557694944
periodic_amplitude,3.1129524806001205
periodic_length_scale,1.665164528287903
periodic_period,0.9998833467513067
periodic_exp_length_scale,111.49763428668675
observation_noise_variance,0.05313568953056348


In [21]:
#Xây dựng hàm dự đoán hậu nghiệm Posterior Gaussian process
#Sử dụng các kernel đã xây dựng và các quan sát đã học
gprm = tfd.GaussianProcessRegressionModel(
    mean_fn=mean_kernel,
    kernel=kernel,
    index_points=X_test.Date.values.reshape(-1, 1),
    observation_index_points=X_train.Date.values.reshape(-1, 1),
    observations=X_train.CO2.values,
    observation_noise_variance=observation_noise_variance)

**Vẽ biểu đồ lấy mẫu dự đoán hậu nghiệm (posterior predictions)**

In [22]:
# Dự đoán hậu nghiệm trung bình hàm hậu nghiệm
gp_mean = gprm.mean().numpy()
# Dự đoán độ lệch chuẩn của kết quả hàm hậu nghiệm
gp_noise = gprm.stddev().numpy()

# X từ 2012 -> 2021
# y từ nồng độ 384 -> 418
fig = bokeh.plotting.figure(
    width=800, height=600,
    x_range=(2012, 2021.3), y_range=(384, 418))

fig.add_layout(bokeh.models.Title(
    text='kết quả dự đoán của Gaussian process regression dựa trên dataset đã huấn luyện cho tập test sau năm 2012.',
    text_font_style="italic"), 'above')

#Định nghĩa các thành phần trong sơ đồ 
fig.xaxis.axis_label = 'Thời gian (Date)'
fig.yaxis.axis_label = 'Lượng CO₂ (ppm)'
fig.add_layout(bokeh.models.Title(text='Nồng độ CO2 trong khí quyển',
                                  text_font_size="14pt"), 'above')

fig.circle(df.Date, df.CO2, legend_label='Các quan sat trên Test data',
           size=2, line_color='darkblue')

fig.line(X_test.Date.values, gp_mean, 
          legend_label='Gaussian process mean function (predictions)',
          line_width=2, line_color='red')

# Vẽ khoảng tin cậy confidence interval (CI)
band_x = np.append(X_test.Date.values, X_test.Date.values[::-1])
band_y = np.append((gp_mean + 2*gp_noise), (gp_mean - 2*gp_noise)[::-1])

fig.patch(band_x, band_y, color='lightblue', alpha=0.4, 
    line_color='lightblue', legend_label='Confidence interval (+/- 2σ noise)')

fig.legend.location = 'top_left'
fig.toolbar.autohide = True
bokeh.plotting.show(fig)